# This notebook contains functions to crop away the black or white defects in images and the function to optimize the contrast and brightness of images

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/') # mount drive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [ ]:
from PIL import Image
import os
import cv2
import numpy as np

In [ ]:
def label_if_defect(file):
    """
    This function classifies those images into 4 classes:
    perfect images(0), with black defect(1), with white defect(2), black & white defect(3)
    """
    img = cv2.imread(file, cv2.IMREAD_GRAYSCALE) # read image file in gray scale
    nb_black = len((np.where(img==0))[0]) # count number of black pixels in this image
    nb_white = len((np.where(img==255))[0]) # count number of white pixels in this image
    nb_pixels = img.shape[0]*img.shape[1]  # count number of pixels in this image

    label = 0
    if nb_black/nb_pixels > 0.1:
        label = 1
    if nb_white/nb_pixels > 0.1:
        if label == 1:
            label = 3
        if label == 0:
            label = 2

    return label
  

In [ ]:
# helper function
def label_if_defect(img,threshold_label=0.1):
    """
    function to classify the images as one with black area (1), with white defect(2), black & white defect(3)
    ---------------------------------------------------------------------------------------------------------
    parameters:
    img: image type variable to detect defects
    threshold_label: the percentage threshold for defining an image with defect

    return:
    label: perfect images(0), with black defect(1), with white defect(2), black & white defect(3)
    """
    nb_black = len((np.where(img==0))[0]) # count number of black pixels in this image
    nb_white = len((np.where(img==255))[0]) # count number of white pixels in this image
    nb_pixels = img.shape[0]*img.shape[1]  # count number of pixels in this image

    label = 0
    if nb_black/nb_pixels > threshold_label:
        label = 1
    if nb_white/nb_pixels > threshold_label:
        if label == 1:
            label = 3
        if label == 0:
            label = 2

    return label

In [ ]:
# helper function
def largest_no_defect_img_white_hor(file,threshold):
    """
    This function crops out the approximately very large useful region of the image if it is filled with white defect
    images are cut only in horizontal direction
    ------------------------------------------------------------------------------------------------------------------
    parameters:
    threshold: number of white pixels allowed in one row.
    """
    img = cv2.imread(file, cv2.IMREAD_GRAYSCALE) # read image file in gray scale
    img2 = img.copy()
    nb_white_row= np.count_nonzero(img==255,axis = 1) # count number of white pixels in each row
    threshold_nb_white = img.shape[1] * threshold
    index_cut_start = 0  # initialize the indexs to cut
    index_cut_end = img.shape[0]-1
    indexes = np.where(nb_white_row <= threshold_nb_white)[0]
    #print(indexes)

    index_cut_start = indexes[0]
    index_cut_end = indexes[-1]
  
    img_final = img2[index_cut_start:index_cut_end]
   # print(img_final.shape)

    return img_final
  

In [ ]:
def largest_no_defect_img_white_ver(file,threshold):
    """
    function to crop out a very large useful region of the image if it is filled with white defect
    images are cut only in vertical direction
  ------------------------------------------------------------------------------------------------------------------
    parameters:
    threshold: number of white pixels allowed in one column.
    """

    img = cv2.imread(file, cv2.IMREAD_GRAYSCALE) # read image file in gray scale
    img2 = img.copy()
    nb_white_row= np.count_nonzero(img==255,axis = 0) # count number of white pixels in each column
    threshold_nb_white = img.shape[0] * threshold


    index_cut_start = 0  # initialize the indexs to cut
    index_cut_end = img.shape[1]-1

    indexes = np.where(nb_white_row <= threshold_nb_white)[0]
    #print(indexes)

    index_cut_start = indexes[0]
    index_cut_end = indexes[-1]
    #print(index_cut_start)
    #print(index_cut_end)

    img_final = img2[:,index_cut_start:index_cut_end]
    print(img_final.shape)

    return img_final

In [ ]:
def largest_no_defect_img_white_recursion_hor(filename,start_threshold,threshold_label,factor):
    """
    This function uses a recursion algorithm to cut the images with large white region horizontally

    parameter:
    start_threshold: starting percentage of white pixels allowed in one row
    threshold_label: allowed percentage of white pixels in an image 
    factor: factor, used when decreasing the percentage of white pixels allowed in one row

    return:
    img: image after cropping
    label: type of the returned image -- perfect images(0), with black defect(1), with white defect(2), black & white defect(3)
    """
    threshold = start_threshold
    iterations = 10

    for i in range(iterations):
        img = largest_no_defect_img_white_hor(filename,threshold)
        label = label_if_defect(img,threshold_label)

        if label !=2:
            break
        else:
            threshold = threshold/factor
    return img,label

In [ ]:
def largest_no_defect_img_white_recursion_ver(filename,start_threshold,threshold_label,factor):
    """
    This function uses a recursion algorithm to cut the images with large white region vertically

    parameter:
    start_threshold: starting percentage of white pixels allowed in one row
    threshold_label: allowed percentage of white pixels in an image 
    factor: factor, used when decreasing the percentage of white pixels allowed in one row

    return:
    img: image after cropping
    label: type of the returned image -- perfect images(0), with black defect(1), with white defect(2), black & white defect(3)
    """
    threshold = start_threshold
    iterations = 10

    for i in range(iterations):
        img = largest_no_defect_img_white_ver(filename,threshold)
        label = label_if_defect(img,threshold_label)

        if label !=2:
            break
        else:
            threshold = threshold/factor
    return img,label

In [ ]:
def largest_no_defect_img_black_hor(file,threshold):
    """
    This function crops out the approximately very large useful region of the image if it is filled with black defect
    images are cut only in horizontal direction
    ------------------------------------------------------------------------------------------------------------------
    parameters:
    threshold: number of black pixels allowed in one row.
    """
    img = cv2.imread(file, cv2.IMREAD_GRAYSCALE) # read image file in gray scale
    img2 = img.copy()
    nb_black_row= np.count_nonzero(img==0,axis = 1) # count number of black pixels in each row
    threshold_nb_black = img.shape[1] * threshold


    index_cut_start = 0  # initialize the indexs to cut
    index_cut_end = img.shape[0]-1

    indexes = np.where(nb_black_row <= threshold_nb_black)[0]
    #print(indexes)

    index_cut_start = indexes[0]
    index_cut_end = indexes[-1]
  
    img_final = img2[index_cut_start:index_cut_end]
    print(img_final.shape)

    return img_final

In [ ]:
def largest_no_defect_img_black_ver(file,threshold):
    """
    function to crop out a very large useful region of the image if it is filled with black defect
    images are cut only in vertical direction
    ------------------------------------------------------------------------------------------------------------------
    parameters:
    threshold: number of black pixels allowed in one column.
    """
    img = cv2.imread(file, cv2.IMREAD_GRAYSCALE) # read image file in gray scale
    img2 = img.copy()
    nb_black_row= np.count_nonzero(img==0,axis = 0) # count number of black pixels in each column
    threshold_nb_black = img.shape[0] * threshold
    index_cut_start = 0  # initialize the indexs to cut
    index_cut_end = img.shape[1]-1
    indexes = np.where(nb_black_row <= threshold_nb_black)[0]
    #print(indexes)
    index_cut_start = indexes[0]
    index_cut_end = indexes[-1]
    #print(index_cut_start)
    #print(index_cut_end)

    img_final = img2[:,index_cut_start:index_cut_end]
    print(img_final.shape)

    return img_final

In [ ]:
def largest_no_defect_img_black_recursion_hor(filename,start_threshold,threshold_label,factor):
    """
    This function uses a recursion algorithm to cut the images with large black region horizontally

    parameter:
    start_threshold: starting percentage of black pixels allowed in one row
    threshold_label: allowed percentage of black pixels in an image 
    factor: factor, used when decreasing the percentage of black pixels allowed in one row

    return:
    img: image after cropping
    label: type of the returned image -- perfect images(0), with black defect(1), with white defect(2), black & white defect(3)
    """
    threshold = start_threshold
    iterations = 10 # maximum number of iterations allowed
    for i in range(iterations):
        img = largest_no_defect_img_black_hor(filename,threshold)
        label = label_if_defect(img,threshold_label)

        if label !=1:
            break
        else:
            threshold = threshold/factor
    return img,label

In [ ]:
def largest_no_defect_img_black_recursion_ver(filename,start_threshold,threshold_label,factor):
    """
    This function uses a recursion algorithm to cut the images with large black region vertically

    parameter:
    start_threshold: starting percentage of black pixels allowed in one row
    threshold_label: allowed percentage of black pixels in an image 
    factor: factor, used when decreasing the percentage of black pixels allowed in one row

    return:
    img: image after cropping
    label: type of the returned image -- perfect images(0), with black defect(1), with white defect(2), black & white defect(3)
    """
    threshold = start_threshold
    iterations = 10 # maximum number of iterations allowed
  
    for i in range(iterations):
        img = largest_no_defect_img_black_ver(filename,threshold)
        label = label_if_defect(img,threshold_label)

        if label !=1:
            break
        else:
            threshold = threshold/factor
    return img,label

In [ ]:
# perfect images(0), with black defect(1), with white defect(2), black & white defect(3)
def cropping_max_area(filename,start_threshold,threshold_label,factor):
    """
    This function do both horizontal cut and vertical cut, black&white defect type images are not considered in this function 
    """

    img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE) # read image file in gray scale
  
    file_type = label_if_defect(img) 
    if (file_type==1):
    # do horizontal recursive cutting first
        img,label = largest_no_defect_img_black_recursion_hor(filename,start_threshold,threshold_label,factor)

    # if image is not a perfect image after horizontal cut, then do vertical cut
        if label ==1:
            img, label = largest_no_defect_img_black_recursion_ver(filename,start_threshold,threshold_label,factor)
    elif (file_type==2):
    # do horizontal recursive cutting first
        img,label = largest_no_defect_img_white_recursion_hor(filename,start_threshold,threshold_label,factor)
    # if image is not a perfect image after horizontal cut, then do vertical cut
        if label ==2:
            img, label = largest_no_defect_img_white_recursion_ver(filename,start_threshold,threshold_label,factor)
    elif (file_type==0):
        label = 0
  
    return img, label


In [ ]:
def auto_optim(original_image):
    """
    This function optimize the contrast and brightness of images using contrast limited adaptive histogram equalization
    """
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    optimised_image = clahe.apply(original_image)
    return optimised_image